### 직무별-테마별 요약

In [3]:
# import libraries
import pandas as pd
import pickle
import torch
import os
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
nltk.download('punkt')

# !pip install transformers[torch]==4.27.4
# !pip install protobuf==3.20

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunga\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# make a folder if it's not exist yet
def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [15]:
# make a folder to save data
folder_path = '.\\result2'
makedirs(folder_path)

data_path = '..\\reviewdataset\\review_posneg_theme_byjob\\'
file_list = os.listdir(data_path)
file_list_name = [file for file in file_list if file.endswith('.pkl')]

for file in file_list_name:
	filename = file.split('.pkl')[0]

	# load a pickle file
	data_test = []

	with open('{}{}'.format(data_path, file), 'rb') as tf:
		data_test = pickle.load(tf)
	tf.close()

	# join list elements in one sentence
	sentence_all = '\n'.join(data_test)

	# model
	tokenizer = AutoTokenizer.from_pretrained("digit82/kobart-summarization")
	model = AutoModelForSeq2SeqLM.from_pretrained("digit82/kobart-summarization")

	# parameters
	max_input_length = 512

	inputs = ["summarize_summary: " + sentence_all]  # summarize

	inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
	output = model.generate(**inputs,
						 do_sample=True,
						 min_length=10,
						 max_length=100,
                         num_beams = 21,  
                         repetition_penalty = 1000.1,
                         temperature = 0.7,
                         top_k = 20,
                         top_p = 0.95,
                         eos_token_id = 1)
	decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
	predicted = nltk.sent_tokenize(decoded_output.strip())[0]

	# save a predicted sentence as a text file
	with open('{}/{}_smr.txt'.format(folder_path,filename),'w') as f:
		f.write(predicted)
	f.close()